<a href="https://colab.research.google.com/github/cwagner2325/WiseInbox_ML/blob/main/Wise_Inbox_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 736.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=255320 sha256=b36fb567d9699cb9fa828974e6c14fb00b7e12bd60ea252bce5f3d5652bf684b
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.2/127.2 kB 14.0 MB/s eta 0:00:00


In [2]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import random
import os
from google.colab import userdata
from pyngrok import conf
from time import sleep
import sklearn
from sklearn.pipeline import Pipeline

In [3]:
app = FastAPI()

In [4]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [5]:
class model_input(BaseModel):

    Email : str
    Sender: str
    Subject : str

In [6]:
with open('/content/drive/MyDrive/XGBModel4', 'rb') as model_file:
    classifier = pickle.load(model_file)

with open('/content/drive/MyDrive/XGBModel4Vectorizer', 'rb') as vectorizer_file:
    vectorizer = pickle.load(vectorizer_file)

In [7]:
def predictEmail(input_text):
  transformed_email = vectorizer.transform([input_text])
  prediction = classifier.predict(transformed_email)
  probabilities = classifier.predict_proba(transformed_email)

  PHISHING = 1;
  SAFE = 0

  if prediction == SAFE:
      print("Predicted class: Safe Email")
      print("Probabilities:", probabilities.item(0))
      return "Safe Email", probabilities.item(0)
  elif prediction == PHISHING:
      print("Predicted class: Phishing Email")
      print("Probabilities:", probabilities.item(1))
      return "Phishing Email", probabilities.item(1)

In [8]:
@app.post('/email_prediction')
def email_prediction(input_parameters : model_input):
    input_data = input_parameters.json()
    input_dictionary = json.loads(input_data)

    emailBody = input_dictionary['Email']
    sender = input_dictionary['Sender']
    subject = input_dictionary['Subject']

    predictionResult, confidence = predictEmail(emailBody)

    if predictionResult == "Phishing Email":
        # If the model is 100% confident it's a phishing email, set the prediction to 50
        prediction = 100 * (1.05 - (confidence / 2))
    else:
        # If the model is 100% confident it's a safe email, set the prediction to 100
        prediction = 100

    return {"prediction": prediction}

In [ ]:
os.environ["TOKEN"] = userdata.get("TOKEN")
ngrok.set_auth_token(os.environ["TOKEN"])

ngrok_tunnel = ngrok.connect(8000)

# print('Public URL:', ngrok_tunnel.public_url)
# print("Here is a test command to run in a terminal: ")
# print('curl --insecure -X POST -H "Content-Type: application/json" -d \'{\n' +
#       '\t"Email": "example@example.com",\n\t"Sender": "John Doe",\n\t"Subject": "Important News" \n}\' ' +
#       ngrok_tunnel.public_url + "/email_prediction")

print("Running at public URL:", ngrok_tunnel.public_url + "/email_prediction")

nest_asyncio.apply()
uvicorn.run(app, port=8000, log_level="warning")

Running at public URL: https://6395-34-145-255-232.ngrok-free.app/email_prediction
Predicted class: Safe Email
Probabilities: 0.8213399052619934
Predicted class: Safe Email
Probabilities: 0.984958291053772
Predicted class: Safe Email
Probabilities: 0.96560138463974
Predicted class: Safe Email
Probabilities: 0.9778555631637573
Predicted class: Safe Email
Probabilities: 0.9706808924674988
Predicted class: Phishing Email
Probabilities: 0.7536976337432861
Predicted class: Safe Email
Probabilities: 0.8923309445381165
Predicted class: Safe Email
Probabilities: 0.9990854859352112
Predicted class: Safe Email
Probabilities: 0.8725208640098572
Predicted class: Safe Email
Probabilities: 0.9126431941986084
